---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd
import numpy as np
# 读取csv格式文件，index_col=0表示让columns=0的这列作为df的index，skiprows=1表示跳过第1行
df = pd.read_csv('http://7xrwo9.com1.z0.glb.clouddn.com/assignment2-olympics.csv', index_col=0, skiprows=1)

# 循环遍历columns里的列名，并替换成合适的名称; len('№') == 1; inplace=True表示更改源文件内容，因为rename()方法会返回一个副本
for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

# '\s\('表示一个空格加上一个左圆括号，根据这个规则对index进行split()处理 
names_ids = df.index.str.split('\s\(') # split the index by '('

# [0]返回空格前面一部分内容，即完整的国家名称
df.index = names_ids.str[0] # the [0] element is the country name (new index) 
# [1]返回左圆括号后面的内容，如果取前3个字符，即得到城市的3字代码，df['ID']添加新的一列到columns，name=ID，values=城市三字代码
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

# 利用drop()丢掉标签为'Totals'这一栏，默认axis=0针对row操作，另一种方法可以在读取csv文件的同时，利用skipfooter=1直接忽略Totals这栏
df = df.drop('Totals')
# 显示df前5栏，head(n)里如果添加参数n,可指定显示n栏
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting

    return df.iloc[0]  #方法一
#     return df.loc['Afghanistan']  #方法二
#     return df.ix[0,:]  #方法三
# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    # 方法一：将df按照'Gold'列排序，默认升序，所以最大值在最后，通过df1.iloc[-1]取到最后一栏(类型为Series)，该Series的name属性对应国家名称
#     df1 = df.sort_values('Gold')
#     s1 = df1.iloc[-1]
#     return s1.name

    # 方法二：将df按照'Gold'列降序排序，该df的index第一个位置即对应的国家名称
#     df1 = df.sort_values('Gold', ascending=False)
#     return df1.index[0]

    # 方法三：通过idxmax()直接返回最大值对应的index
    return df['Gold'].idxmax()
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    # 利用Series的广播特性，计算'Gold'和'Gold.1'差的绝对值，然后运用idmax()返回对应的index
    ans = abs(df['Gold']-df['Gold.1'])
    return ans.idxmax()
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    # 筛选出夏季和冬季奥运会金牌数都大于等于1
    df3 = df[(df['Gold'] >= 1) & (df['Gold.1'] >= 1)]
    # 利用公式求值，并取绝对值
    ans = abs((df3['Gold'] - df3['Gold.1']) / df['Gold.2'])
    # 返回最大值对应的index
    return ans.idxmax()
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [6]:
def answer_four():
    # 为了不改变df，先复制一个df4
    df4 = df.copy()
    # 添加新列'Points'，值为总金牌数*3 + 总银牌数*2 + 总铜牌数*1
    df4['Points'] = df4['Gold.2']*3 + df4['Silver.2']*2 + df4['Bronze.2']*1
    return df4['Points']
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.


### The key for SUMLEV is as follows:

- **40 = State** and/or Statistical Equivalent 
- **50 = County** and /or Statistical Equivalent

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [7]:
census_df = pd.read_csv('http://7xrwo9.com1.z0.glb.clouddn.com/assignment2-census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [8]:
def answer_five():
    # 根据提示(SUMLEV'列下的County对应50、State对应40)，筛选出只包含50的新df
    df5 = census_df[census_df['SUMLEV'] == 50]
    # 通过groupby()对'STNAME'进行分组并计算'CTYNAME'的数量，返回的是一个index为'STNAME'、name为'CTYNAME'的Series
    ans = df5.groupby('STNAME')['CTYNAME'].count()
    # 利用idxmax()求出最大值对应的index
    return ans.idxmax()
answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [9]:
def answer_six():
    # 筛选出只包含50的新df
    df6 = census_df[census_df['SUMLEV'] == 50]
    # 找出每个state人口最多的3个counties, nlargest(3)可以筛选出最大的3个值(得到的是一个Series)
    # 通过reset_index()将series转换为DataFrame,为下一步继续groupby做准备
    df_top_country = df6.groupby('STNAME')['CENSUS2010POP'].nlargest(3).reset_index() # 一个疑惑，为什么groupby后会变成多层index，而下面的groupby没有
    # 对上一步得到的df进行groupby，并运用sum()，然后筛选出Series中最大的三个值(可以用降序排列,通过head(3)得到；也可以通过nlargest(3)求得)
    df_top_state = df_top_country.groupby('STNAME')['CENSUS2010POP'].sum().nlargest(3)
    # 利用tolist()将index转换为list
    ans = df_top_state.index.tolist()
    return ans
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [10]:
def answer_seven():
    # 复制一个census_df副本
    df7 = census_df.copy()
    # 筛选出只包含50的新df
    df7 = df7[df7['SUMLEV'] == 50]
    # 求出每一栏中'POPESTIMATE2010—POPESTIMATE2015'6列中的最大值和最小值(axis=1横着计算)
    maxx = df7[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)
    minx = df7[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    # 利用max和min差的绝对值给df添加新列'ABS_CHANGE'
    df7['ABS_CHANGE'] = abs(maxx-minx)
    # 根据'ABS_CHANGE'列的values值进行降序排列
    df7_sorted = df7.sort_values('ABS_CHANGE', ascending=False)
    # 新排序的df对应的'CTYNAME'列的values的第一个值就是要求的值
    ans = df7_sorted['CTYNAME'].values[0]
    return ans
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [11]:
def answer_eight():
    df8 = census_df.copy() # 复制一个census_df的副本
    df8 = df8[(df8['REGION'] == 1) | (df8['REGION'] == 2)] #筛选出REGION是1或2
    df8 = df8[df8['CTYNAME'].str.startswith('Washington')] #筛选出CTYNAME是'Washington'开头的
    df8 = df8[df8['POPESTIMATE2015'] > df8['POPESTIMATE2014']] #筛选出POPESTIMATE2015大于POPESTIMATE 2014
    df_ans = df8[['STNAME', 'CTYNAME']].sort_index().head() # 创建一个5x2，以['STNAME', 'CTYNAME']为columns，按照indexs升序排列的df
    return df_ans
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
